### Day 12

It might be easier to debug this one with a notebook.

In [1]:
from collections import namedtuple
from pyrsistent import pvector
import itertools as iter
import operator as op


Vector = namedtuple('Vector', 'x y z')
Moon = namedtuple('Moon', 'position velocity')


def vmap(f, *args):
    'Element-wise map for Vectors'
    return Vector(f(*[v.x for v in args]), 
                  f(*[v.y for v in args]),
                  f(*[v.z for v in args]))


def parse_moons(s):
    result = pvector()
    
    for line in s.strip().splitlines():
        position = Vector(*[int(x.split('=')[1]) for x in line.strip().strip('<>').split(',')])
        velocity = Vector(0, 0, 0)
        result = result.append(Moon(position, velocity))
        
    return result

In [2]:
test_input = '''<x=-1, y=0, z=2>
<x=2, y=-10, z=-7>
<x=4, y=-8, z=8>
<x=3, y=5, z=-1>'''

test_system = parse_moons(test_input)
test_system

pvector([Moon(position=Vector(x=-1, y=0, z=2), velocity=Vector(x=0, y=0, z=0)), Moon(position=Vector(x=2, y=-10, z=-7), velocity=Vector(x=0, y=0, z=0)), Moon(position=Vector(x=4, y=-8, z=8), velocity=Vector(x=0, y=0, z=0)), Moon(position=Vector(x=3, y=5, z=-1), velocity=Vector(x=0, y=0, z=0))])

In [3]:
def unit_sign(x):
    if x == 0: 
        return 0
    elif x < 0:
        return -1
    else:
        return 1

    
def simulate(system):
    # For each non-matching pair in either direction
    for i, j in iter.permutations(range(len(system)), 2):
        # Difference between moons' positions
        d = vmap(op.sub, system[j].position, system[i].position)
        # Difference changed to unit values or zero
        unit_d = vmap(unit_sign, d)
        # Add to the velocity
        new_velocity = vmap(op.add, system[i].velocity, unit_d)
        system = system.set(i, system[i]._replace(velocity=new_velocity))
        
    # Update each moon with its new position
    for i, moon in enumerate(system):
        system = system.set(i, Moon(vmap(op.add, moon.position, moon.velocity), 
                                    moon.velocity))
    
    return system


def print_system(system):
    for moon in system:
        print(f'pos=<x={moon.position.x}, y={moon.position.y}, z={moon.position.z}>, '
              f'vel=<x={moon.velocity.x}, y={moon.velocity.y}, z={moon.velocity.z}>')

In [4]:
ts = test_system

for i in range(11):
    print(f'After {i} steps:')
    print_system(ts)
    print()
    
    ts = simulate(ts)

After 0 steps:
pos=<x=-1, y=0, z=2>, vel=<x=0, y=0, z=0>
pos=<x=2, y=-10, z=-7>, vel=<x=0, y=0, z=0>
pos=<x=4, y=-8, z=8>, vel=<x=0, y=0, z=0>
pos=<x=3, y=5, z=-1>, vel=<x=0, y=0, z=0>

After 1 steps:
pos=<x=2, y=-1, z=1>, vel=<x=3, y=-1, z=-1>
pos=<x=3, y=-7, z=-4>, vel=<x=1, y=3, z=3>
pos=<x=1, y=-7, z=5>, vel=<x=-3, y=1, z=-3>
pos=<x=2, y=2, z=0>, vel=<x=-1, y=-3, z=1>

After 2 steps:
pos=<x=5, y=-3, z=-1>, vel=<x=3, y=-2, z=-2>
pos=<x=1, y=-2, z=2>, vel=<x=-2, y=5, z=6>
pos=<x=1, y=-4, z=-1>, vel=<x=0, y=3, z=-6>
pos=<x=1, y=-4, z=2>, vel=<x=-1, y=-6, z=2>

After 3 steps:
pos=<x=5, y=-6, z=-1>, vel=<x=0, y=-3, z=0>
pos=<x=0, y=0, z=6>, vel=<x=-1, y=2, z=4>
pos=<x=2, y=1, z=-5>, vel=<x=1, y=5, z=-4>
pos=<x=1, y=-8, z=2>, vel=<x=0, y=-4, z=0>

After 4 steps:
pos=<x=2, y=-8, z=0>, vel=<x=-3, y=-2, z=1>
pos=<x=2, y=1, z=7>, vel=<x=2, y=1, z=1>
pos=<x=2, y=3, z=-6>, vel=<x=0, y=2, z=-1>
pos=<x=2, y=-9, z=1>, vel=<x=1, y=-1, z=-1>

After 5 steps:
pos=<x=-1, y=-9, z=2>, vel=<x=-3, y=-1, z

In [5]:
def potential_energy(moon):
    return (sum(abs(x) for x in moon.position)
            * sum(abs(x) for x in moon.velocity))


def total_energy(system):
    return sum(potential_energy(moon) for moon in system)


def run_n(f, x, n):
    while n > 0:
        x = f(x)
        n -= 1
        
    return x

In [6]:
total_energy(run_n(simulate, test_system, 10))

179

In [7]:
test_input_2 = '''<x=-8, y=-10, z=0>
<x=5, y=5, z=10>
<x=2, y=-7, z=3>
<x=9, y=-8, z=-3>'''

test_system_2 = parse_moons(test_input_2)

total_energy(run_n(simulate, test_system_2, 100))

1940

In [24]:
system = parse_moons(open('input', 'r').read())

total_energy(run_n(simulate, system, 1000))

14809

#### Part 2

The $x$, $y$ and $z$ components are independent, so note where they cycle.

In [16]:
def vmap1D(f, *args):
    return f(*args)


def simulate1D(system):
    # For each non-matching pair in either direction
    for i, j in iter.permutations(range(len(system)), 2):
        # Difference between moons' positions
        d = vmap1D(op.sub, system[j].position, system[i].position)
        # Difference changed to unit values or zero
        unit_d = vmap1D(unit_sign, d)
        # Add to the velocity
        new_velocity = vmap1D(op.add, system[i].velocity, unit_d)
        system = system.set(i, system[i]._replace(velocity=new_velocity))
        
    # Update each moon with its new position
    for i, moon in enumerate(system):
        system = system.set(i, Moon(vmap1D(op.add, moon.position, moon.velocity), 
                                    moon.velocity))
    
    return system


def find_cycle1D(system_dim):
    seen = {system_dim: 0}
    t = 0
    while True:
        t += 1
        system_dim = simulate1D(system_dim)
        if system_dim in seen:
            return (seen[system_dim], t)
        seen[system_dim] = t

        
def find_cycles(system):
    system_x = pvector(Moon(moon.position.x, moon.velocity.x) for moon in system)
    system_y = pvector(Moon(moon.position.y, moon.velocity.y) for moon in system)
    system_z = pvector(Moon(moon.position.z, moon.velocity.z) for moon in system)
    return (find_cycle1D(system_x), find_cycle1D(system_y), find_cycle1D(system_z))

In [17]:
find_cycles(test_system)

((0, 18), (0, 28), (0, 44))

These start at zero, which would make things easier. Do they all start at zero?

In [19]:
find_cycles(test_system_2)

((0, 2028), (0, 5898), (0, 4702))

In [20]:
find_cycles(system)

((0, 167624), (0, 231614), (0, 116328))

Yes. Assuming that's always the case, which I think it might be as they'll need to loop back to get to where they were, then find the least common multiple of the cycles.

In [35]:
import math
from functools import reduce


def find_system_cycle(system):
    cycles = find_cycles(system)
    assert all(x[0] == 0 for x in cycles)
    loop_ends = [x[1] for x in cycles]
    return reduce(lambda x, y: x * y // math.gcd(x, y), loop_ends)
    


find_system_cycle(test_system)

2772

In [37]:
find_system_cycle(test_system_2)

4686774924

In [39]:
%%time

find_system_cycle(system)

CPU times: user 19.3 s, sys: 216 ms, total: 19.5 s
Wall time: 19.6 s


282270365571288